In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install torch 
!pip install transformers
!pip install datasets 
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13

In [1]:
# REFERENCIAS: 

# paper captum library: https://arxiv.org/pdf/2009.07896.pdf 
# paper Integrated Gradients: https://arxiv.org/pdf/1703.01365.pdf



In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd 
import numpy as np 

import captum
from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

from datasets import Dataset
import torch
import torch.nn as nn


In [6]:
print(captum.__version__)

0.6.0


In [7]:
# load test 
df_test = pd.read_xml("/content/drive/MyDrive/ICMC/Introducao-PLN/atividades/exercicio-03/project_embeddings/dados_test_tratados.xml")
df_test.head()

,text,label
0,o cachorro caramelo está assistindo um cachorr...,0
1,o cara está fazendo exercícios no chão [sep] u...,1
2,um cachorro grande e um cachorro pequenino est...,1
3,um menino jovem vestindo um traje de banho ver...,0
4,um cara velho com uma barba que é cinza está a...,1


In [8]:
print(torch.cuda.is_available())

True


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [10]:
# Carregando o tokenizer e o modelo do BERT pré-treinado em português
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
# carrega modelo 
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/ICMC/Introducao-PLN/atividades/exercicio-03/project_embeddings/bertimbau-base-fine-tune')
model.to(device)
model.eval()
model.zero_grad()

In [11]:
model.device

device(type='cuda', index=0)

In [12]:
# text to inference 
text = ["'Uma garrafa está sendo lambida pelo gato [sep] O gato está lambendo um objeto'"]
# tokeniza dados
inputs = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

In [13]:
inputs = inputs.to(device)

In [14]:
model.bert.embeddings.word_embeddings

Embedding(29794, 768, padding_idx=0)

In [15]:
# Faz a inferência do modelo
outputs = model(**inputs)[0]
prediction = torch.argmax(outputs).item()
print("Prediction:", prediction)

# Define a função de linha de base como uma sequência de zeros
baseline = torch.zeros_like(inputs['input_ids'])

Prediction: 1


In [16]:
# Define model output
def model_output(inputs):
  return model(inputs)[0]

# Define model input
model_input = model.bert.embeddings

In [17]:
# Integrated Gradients 
lig = LayerIntegratedGradients(model_output, model_input)

In [18]:
def construct_input_and_baseline(text):

    max_length = 128
    baseline_token_id = tokenizer.pad_token_id 
    sep_token_id = tokenizer.sep_token_id 
    cls_token_id = tokenizer.cls_token_id 

    text_ids = tokenizer.encode(text, max_length=max_length, truncation=True, add_special_tokens=False)
   
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    token_list = tokenizer.convert_ids_to_tokens(input_ids)

    baseline_input_ids = [cls_token_id] + [baseline_token_id] * len(text_ids) + [sep_token_id]
    return torch.tensor([input_ids], device=device), torch.tensor([baseline_input_ids], device=device), token_list

text = 'Uma garrafa está sendo lambida pelo gato [sep] O gato está lambendo um objeto'
input_ids, baseline_input_ids, all_tokens = construct_input_and_baseline(text)

print(f'original text: {input_ids}')
print(f'baseline text: {baseline_input_ids}')

original text: tensor([[  101,  1431, 16317,  1165,   698,   660, 19068,  3301,   285,   423,
         15997,   164, 21333,   166,   231, 15997,   698, 19068, 21954,   222,
          4947,   102]], device='cuda:0')
baseline text: tensor([[101,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0, 102]], device='cuda:0')


In [19]:
input_ids.shape, baseline_input_ids.shape

(torch.Size([1, 22]), torch.Size([1, 22]))

In [20]:
# Faz a inferência do modelo
outputs = model(**inputs)
logits = outputs[0]

# Define o target para a classe prevista
prediction = torch.argmax(logits).item()
target = torch.tensor(prediction).unsqueeze(0)

In [21]:
attributions, delta = lig.attribute(inputs= input_ids,
                                    baselines= baseline_input_ids,
                                    target=target,
                                    return_convergence_delta=True,
                                    internal_batch_size=1
                                    )
print(attributions.size())

torch.Size([1, 22, 768])


In [22]:
# sumarizar atribuicoes 
def summarize_attributions(attributions):

    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    
    return attributions

attributions_sum = summarize_attributions(attributions)
print(attributions_sum.size())

torch.Size([22])


In [23]:
from captum.attr import visualization as viz

score_vis = viz.VisualizationDataRecord(
                        word_attributions = attributions_sum,
                        pred_prob = torch.max(model(input_ids)[0]),
                        pred_class = torch.argmax(model(input_ids)[0]).cpu().numpy(),
                        true_class = 1,
                        attr_class = text,
                        attr_score = attributions_sum.sum(),       
                        raw_input_ids = all_tokens,
                        convergence_score = delta)

In [24]:
viz.visualize_text([score_vis])

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (2.50),Uma garrafa está sendo lambida pelo gato [sep] O gato está lambendo um objeto,1.41,[CLS] Uma garra ##fa está sendo lam ##bi ##da pelo gato [ sep ] O gato está lam ##bendo um objeto [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (2.50),Uma garrafa está sendo lambida pelo gato [sep] O gato está lambendo um objeto,1.41,[CLS] Uma garra ##fa está sendo lam ##bi ##da pelo gato [ sep ] O gato está lam ##bendo um objeto [SEP]


In [25]:
# funcao de interpretacao  
def interpret_text(text, true_class):

    input_ids, baseline_input_ids, all_tokens = construct_input_and_baseline(text)
    attributions, delta = lig.attribute(inputs= input_ids,
                                    baselines= baseline_input_ids,
                                    target=true_class,
                                    return_convergence_delta=True,
                                    internal_batch_size=1
                                    )
    attributions_sum = summarize_attributions(attributions)

    score_vis = viz.VisualizationDataRecord(
                        word_attributions = attributions_sum,
                        pred_prob = torch.max(model(input_ids)[0]),
                        pred_class = torch.argmax(model(input_ids)[0]).cpu().numpy(),
                        true_class = true_class,
                        attr_class = text,
                        attr_score = attributions_sum.sum(),       
                        raw_input_ids = all_tokens,
                        convergence_score = delta)

    viz.visualize_text([score_vis])

In [26]:
text = "As pessoas não estão andando na estrada ao lado de uma bela cachoeira [sep] Uma cachoeira está fluindo em uma piscina rasa"
true_class = 0
interpret_text(text, true_class)

In [27]:
text = "Quatro pessoas estão andando através da neve espessa e o sol está se pondo [sep] Ninguém está atravessando a neve da linda paisagem nevada"
true_class = 0
interpret_text(text, true_class)

In [28]:
text = "O cara está cuidadosamente alimentando a cobra com um rato. Um cara está alimentando um rato para a cobra"
true_class = 1
interpret_text(text, true_class)

<br>
<br>
<hr>